In [ ]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [ ]:
client.list_experiments()

In [3]:
client.create_experiment(name="my-cool-experiment")

'3'

In [ ]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=10,
    order_by=["metrics.rmse ASC"]
)

In [7]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: f16b59b20c8342aa9592fde07976af7d, rmse: 6.2961
run id: 7a46c9cd4dde45f2a1c3919a0e97dc2d, rmse: 6.2961
run id: cd0a40c6bf8c4f72ac6bf73f95638b9d, rmse: 6.2963
run id: 7fef5a50ddd04b4d86a9ea2c50037da5, rmse: 6.3027
run id: 97eef540368b4353a8c29596db00e7fe, rmse: 6.3071


In [ ]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI